In [5]:
# ==========================================================
# Célula 1: Requisição e criação do DataFrame
# ==========================================================
# Importando as bibliotecas necessárias
import requests
import pandas as pd
from datetime import datetime
import pytz

# Definindo a URL da API REST Countries (com fields)
url = (
    "https://restcountries.com/v3.1/all?"
    "fields=cca2,name,capital,region,subregion,population,area,languages,currencies,flags"
)
print(f"URL da API: {url}")

# Fazendo a requisição para a API
try:
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers, timeout=10)

    if response.status_code == 200:
        print("Requisição feita com sucesso!")
        countries_data = response.json()
        print(f"Total de países retornados: {len(countries_data)}")

        countries_list = []

        for country in countries_data:
            nome_resumido = country.get('cca2', 'Não informado')
            name = country.get("name", {}).get("common", "Não informado")
            official_name = country.get("name", {}).get("official", "Não informado")
            capital = country.get("capital", ["Não informado"])[0] if country.get("capital") else "Não informado"
            region = country.get("region", "Não informado")
            subregion = country.get("subregion", "Não informado")
            population = country.get("population", 0)
            area = country.get("area", 0)
            languages = ", ".join(country.get("languages", {}).values()) if country.get("languages") else "Não informado"
            currencies = ", ".join([v.get("name") for v in country.get("currencies", {}).values()]) if country.get("currencies") else "Não informado"
            flag = country.get("flags", {}).get("png", "Não informado")

            countries_list.append((
            nome_resumido, name, official_name, capital,
            region, subregion, population, area,
            languages, currencies, flag
            ))

        colunas = [
            "nome_resumido", "nome", "nome_oficial", "capital",
            "regiao", "subregiao", "populacao", "area_km2",
            "idiomas", "moedas", "bandeira_url"
        ]

        # A CORREÇÃO ESTÁ AQUI: O NOME DA VARIÁVEL FOI ALTERADO PARA 'df_countries'
        df_countries = pd.DataFrame(countries_list, columns=colunas)

        # Incluindo data de carga
        fuso_brasil = pytz.timezone("America/Sao_Paulo")
        agora_brasil = datetime.now(fuso_brasil)
        df_countries['data_carga'] = agora_brasil.strftime('%Y-%m-%d %H:%M:%S')

        # Exibindo os 10 primeiros países
        display(df_countries.head(10))
        print("df_countries criado com sucesso!")

    else:
        print(f"Erro na requisição. Status code: {response.status_code}")
        print("Resposta do servidor (início):", response.text[:500])

except Exception as e:
    print("Erro durante a requisição:", e)

# ==========================================================
# Célula 2: A consulta SQL
# ==========================================================
#!pip install pandasql
from pandasql import sqldf

# Define uma função para executar consultas SQL
pysqldf = lambda q: sqldf(q, globals())

consultaSQL = """
SELECT *
FROM df_countries 
-- WHERE nome = 'Brazil'
ORDER BY area_km2 DESC
"""

resultado = pysqldf(consultaSQL)
display(resultado)

print("✅ Consulta SQL executada com sucesso!")

URL da API: https://restcountries.com/v3.1/all?fields=cca2,name,capital,region,subregion,population,area,languages,currencies,flags
Requisição feita com sucesso!
Total de países retornados: 250


,nome_resumido,nome,nome_oficial,capital,regiao,subregiao,populacao,area_km2,idiomas,moedas,bandeira_url,data_carga
0,JM,Jamaica,Jamaica,Kingston,Americas,Caribbean,2961161,10991.0,"English, Jamaican Patois",Jamaican dollar,https://flagcdn.com/w320/jm.png,2025-09-25 19:17:29
1,KM,Comoros,Union of the Comoros,Moroni,Africa,Eastern Africa,869595,1862.0,"Arabic, French, Comorian",Comorian franc,https://flagcdn.com/w320/km.png,2025-09-25 19:17:29
2,ZW,Zimbabwe,Republic of Zimbabwe,Harare,Africa,Southern Africa,14862927,390757.0,"Chibarwe, English, Kalanga, Khoisan, Ndau, Nor...",Zimbabwean dollar,https://flagcdn.com/w320/zw.png,2025-09-25 19:17:29
3,SX,Sint Maarten,Sint Maarten,Philipsburg,Americas,Caribbean,40812,34.0,"English, French, Dutch",Netherlands Antillean guilder,https://flagcdn.com/w320/sx.png,2025-09-25 19:17:29
4,LA,Laos,Lao People's Democratic Republic,Vientiane,Asia,South-Eastern Asia,7275556,236800.0,Lao,Lao kip,https://flagcdn.com/w320/la.png,2025-09-25 19:17:29
5,LV,Latvia,Republic of Latvia,Riga,Europe,Northern Europe,1901548,64559.0,Latvian,Euro,https://flagcdn.com/w320/lv.png,2025-09-25 19:17:29
6,TJ,Tajikistan,Republic of Tajikistan,Dushanbe,Asia,Central Asia,9537642,143100.0,"Russian, Tajik",Tajikistani somoni,https://flagcdn.com/w320/tj.png,2025-09-25 19:17:29
7,GB,United Kingdom,United Kingdom of Great Britain and Northern I...,London,Europe,Northern Europe,67215293,242900.0,English,British pound,https://flagcdn.com/w320/gb.png,2025-09-25 19:17:29
8,NI,Nicaragua,Republic of Nicaragua,Managua,Americas,Central America,6624554,130373.0,Spanish,Nicaraguan córdoba,https://flagcdn.com/w320/ni.png,2025-09-25 19:17:29
9,SD,Sudan,Republic of the Sudan,Khartoum,Africa,Northern Africa,43849269,1886068.0,"Arabic, English",Sudanese pound,https://flagcdn.com/w320/sd.png,2025-09-25 19:17:29


df_countries criado com sucesso!


,nome_resumido,nome,nome_oficial,capital,regiao,subregiao,populacao,area_km2,idiomas,moedas,bandeira_url,data_carga
0,RU,Russia,Russian Federation,Moscow,Europe,Eastern Europe,144104080,17098242.00,Russian,Russian ruble,https://flagcdn.com/w320/ru.png,2025-09-25 19:17:29
1,AQ,Antarctica,Antarctica,Não informado,Antarctic,,1000,14000000.00,Não informado,Não informado,https://flagcdn.com/w320/aq.png,2025-09-25 19:17:29
2,CA,Canada,Canada,Ottawa,Americas,North America,38005238,9984670.00,"English, French",Canadian dollar,https://flagcdn.com/w320/ca.png,2025-09-25 19:17:29
3,CN,China,People's Republic of China,Beijing,Asia,Eastern Asia,1402112000,9706961.00,Chinese,Chinese yuan,https://flagcdn.com/w320/cn.png,2025-09-25 19:17:29
4,US,United States,United States of America,"Washington, D.C.",Americas,North America,329484123,9372610.00,English,United States dollar,https://flagcdn.com/w320/us.png,2025-09-25 19:17:29
...,...,...,...,...,...,...,...,...,...,...,...,...
245,CC,Cocos (Keeling) Islands,Territory of the Cocos (Keeling) Islands,West Island,Oceania,Australia and New Zealand,544,14.00,English,Australian dollar,https://flagcdn.com/w320/cc.png,2025-09-25 19:17:29
246,TK,Tokelau,Tokelau,Fakaofo,Oceania,Polynesia,1411,12.00,"English, Samoan, Tokelauan",New Zealand dollar,https://flagcdn.com/w320/tk.png,2025-09-25 19:17:29
247,GI,Gibraltar,Gibraltar,Gibraltar,Europe,Southern Europe,33691,6.00,English,Gibraltar pound,https://flagcdn.com/w320/gi.png,2025-09-25 19:17:29
248,MC,Monaco,Principality of Monaco,Monaco,Europe,Western Europe,39244,2.02,French,Euro,https://flagcdn.com/w320/mc.png,2025-09-25 19:17:29


✅ Consulta SQL executada com sucesso!


In [6]:
#!pip install pandasql
from pandasql import sqldf

# Define uma função para executar consultas SQL
pysqldf = lambda q: sqldf(q, globals())

In [7]:
df_export = pd.DataFrame(resultado)  # Dados dim pais
df_export.to_csv("df_fato_pais.csv", index=False)

In [8]:
# ==========================================================
# Importando bibliotecas necessárias
# ==========================================================
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv, find_dotenv

# Carrega as variáveis de ambiente a partir do arquivo .env
load_dotenv(find_dotenv())

# ==========================================================
# Definindo as credenciais do banco (Render)
# ==========================================================
host = os.getenv("HOST")
port = int(os.getenv("PORT"))
database = os.getenv("DATABASE")
user = os.getenv("USER")
password = os.getenv("PASSWORD")
# ==========================================================
# Criando a conexão com o PostgreSQL usando SQLAlchemy
# ==========================================================
engine = create_engine(
    f"postgresql://{user}:{password}@{host}:{port}/{database}"
)

# ==========================================================
# Salvando no banco (tabela: paises)
# if_exists:
#   - 'replace' -> apaga e recria a tabela
#   - 'append' -> adiciona os dados
# ==========================================================
df_export.to_sql("fato_pais", engine, if_exists="replace", index=False)

print("✅ Dados salvos no banco Render com sucesso!")

✅ Dados salvos no banco Render com sucesso!


In [9]:
#pip install psycopg2
import psycopg2

# Parâmetros de conexão
dbname   = 'projeto_paises'
user     = 'postgres'
password = '252500'
host     = 'localhost'
port     = '5432' 

# Criar uma conexão
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)


cur = conn.cursor()# Criar um cursor  deixa manipular os dados
cur.execute("DELETE FROM TBL_FATO_PAIS")

# executa comandos sql
for indice, coluna_df in resultado.iterrows():
	cur.execute('''
				
				insert into TBL_FATO_PAIS (     
				nome_resumido,
				nome,
				nome_oficial,
				capital,
				regiao,
				subregiao,
				populacao,
				area_km2,
				idiomas,
				moedas,
				bandeira_url,
			 	data_carga
			) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
				
			 ''',(
				 coluna_df["nome_resumido"],
				 coluna_df["nome"],
				 coluna_df["nome_oficial"],
				 coluna_df["capital"],
				 coluna_df["regiao"],
				 coluna_df["subregiao"],
				 coluna_df["populacao"],
				 coluna_df["area_km2"],
				 coluna_df["idiomas"],
				 coluna_df["moedas"],
				 coluna_df["bandeira_url"],
				 coluna_df["data_carga"]
			 )			 
			 )

#comandos sql transct-sql
conn.commit() # validar alterações que agente fez  e subir para o banco de dados 

# Fechar o cursor e a conexão
cur.close()
conn.close()